In [1]:
%connect_info

{
  "kernel_name": "",
  "hb_port": 50913,
  "iopub_port": 45207,
  "stdin_port": 55133,
  "key": "2fe07001-3ece52078c41381229c6f6b5",
  "signature_scheme": "hmac-sha256",
  "transport": "tcp",
  "shell_port": 52735,
  "ip": "127.0.0.1",
  "control_port": 53783
}

Paste the above JSON into a file, and connect with:
    $> jupyter <app> --existing <file>
or, if you are local, you can connect with just:
    $> jupyter <app> --existing kernel-580cbe18-b52e-48f5-886d-73bedbc39567.json
or even just:
    $> jupyter <app> --existing
if this is the most recent Jupyter kernel you have started.


In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib notebook

In [3]:
import json
import matplotlib.patches
import matplotlib.pyplot as plt
import numpy as np

from lieroy import se3
from recov.registration_algorithm import IcpAlgorithm
from recov.censi import registration_and_censi_estimate_from_points
from recov.datasets import create_registration_dataset
import recova.covariance
import recova.clustering
from recova.descriptor.factory import descriptor_factory
from recova.learning.learning import model_from_file
from recova.registration_result_database import RegistrationPairDatabase

In [4]:
dataset = create_registration_dataset('ethz', '~/dataset/ethz/wood_summer')
database = RegistrationPairDatabase('/home/dlandry/dataset/db_eth_05/')

with open('/home/dlandry/dataset/learning_sets/2018-08-10-wood.json') as f:
    learning_set = json.load(f)

with open('/home/dlandry/dataset/learning_runs/2018-08-13-wood-summer.json') as f:
    learning_run = json.load(f)
    
with open('/home/dlandry/dataset/toto.json') as f:
    censi_estimates = json.load(f)
    
model = model_from_file('/home/dlandry/dataset/learning_runs/2018-08-13-wood-summer.model', 'cello')

icp_algo = IcpAlgorithm()
icp_algo.estimate_dist_type = 'normal'
icp_algo.initial_estimate_covariance = 0.05
icp_algo.initial_estimate_covariance_rot = 0.05

In [5]:
descriptor_algo = descriptor_factory(learning_run['metadata']['descriptor_config'])

clustering = recova.clustering.CenteredClusteringAlgorithm(0.005, 100)
covariance_algo = recova.covariance.SamplingCovarianceComputationAlgorithm(clustering)
censi_algo = recova.covariance.CensiCovarianceComputationAlgorithm()

In [6]:
all_pairs = database.registration_pairs()

pairs = list(filter(lambda x: x.dataset == 'wood_summer' and x.reference == x.reading - 1, all_pairs))

In [ ]:
len(pairs)

In [ ]:
censi_covariances = np.empty((len(pairs), 6, 6))
icp_results = np.empty((len(pairs), 4, 4))

for i, pair in enumerate(pairs):
    print('{}/{}'.format(i, len(pairs)))
    t, covariance = registration_and_censi_estimate_from_points(pair.points_of_reading(), pair.points_of_reference(), pair.initial_estimate(), icp_algo)
    censi_covariances[i] = covariance
    icp_results[i] = t

In [ ]:
descriptors = np.zeros((len(pairs), len(descriptor_algo.labels())))

for i, pair in enumerate(pairs):
    descriptors[i] = descriptor_algo.compute(pair)

predictions = model.predict(descriptors)

In [ ]:
def build_trajectory(ts):
    trajectory = np.empty((len(ts) + 1, 4, 4))
    trajectory[0] = np.identity(4)
    for i in range(len(ts)):
        trajectory[i+1] = ts[i] @ trajectory[i]
        
    return trajectory

In [ ]:
icp_trajectory = build_trajectory(icp_results)
icp_trajectory.shape

In [ ]:
ground_truth = np.empty((dataset.n_clouds(), 4, 4))
for i in range(len(ground_truth) - 1):
    ground_truth[i] = dataset.ground_truth(i+1, i)
    
gt_trajectory = build_trajectory(ground_truth)
gt_trajectory.shape

In [ ]:
def plot_trajectory(trajectory, ax, color=None):
    ax.plot(trajectory[:,0,3], trajectory[:,1,3], color=color)


In [ ]:
def mahalanobis(diff, covariance):
    return np.sqrt(diff @ (np.linalg.inv(covariance) @ diff))

In [ ]:
def plot_covariance(mean, covariance, ax):
    eigvals, eigvecs = np.linalg.eig(covariance[0:2,0:2])
    angle = np.arctan2(eigvecs[0,0], eigvecs[0,1]) * 360 / (2 * np.pi)
    width, height = 3 * np.sqrt(eigvals)

    ellipse = matplotlib.patches.Ellipse(xy=mean[0:2,3], width=width, height=height, angle=angle * 360 / (2 * np.pi), fill=False)
    ax.add_artist(ellipse)
    
    

In [ ]:
cum_censi_estimate = np.cumsum(censi_covariances, axis=0)
cum_cello_estimate = np.cumsum(predictions, axis=0)

In [ ]:
fig, ax = plt.subplots()
ax.set_aspect('equal')

plot_trajectory(icp_trajectory, ax, color='0.5')
plot_trajectory(gt_trajectory, ax, color='0.0')

for i in range(len(icp_trajectory) - 1):
    plot_covariance(icp_trajectory[i], cum_cello_estimate[i], ax)


plt.show()

In [ ]:


censi_mahalanobis = np.empty(len(icp_trajectory))
cello_mahalanobis = np.empty(len(icp_trajectory))
for i in range(len(icp_trajectory)):
    diff = np.linalg.inv(icp_trajectory[i]) @ gt_trajectory[i]
    diff_algebra = se3.log(diff)
    
    inv_cov = np.linalg.inv(cum_cello_estimate)
    
    cello_mahalanobis[i] = mahalanobis(diff_algebra, cum_cello_estimate[i])
    censi_mahalanobis[i] = mahalanobis(diff_algebra, cum_censi_estimate[i])

In [ ]:
fig, ax = plt.subplots()

ax.plot(list(range(len(cello_mahalanobis))), cello_mahalanobis)
#plt.plot(list(range(len(censi_mahalanobis))), censi_mahalanobis)

In [ ]:
plt.show()